In [1]:
# Not renaming the file manually to push my skills
# Parallel process iamges to reduce time taken
# save and delete due to space constraints
# Realization accuracy is a bad measure ( dummy )
# Defining custom metric macro F1 for imbalanced data
# Class weights and SMOTE and ADAM to deal with imbalance
# Why SMOTE only for training data
# Changed RElu to ELU
# Changed Macro F1 class from tf to numpy since im using only CPU
# Changed ADAM to NADAM
# Changed Macro F1 class back to tf since im dealing with tensors (lol)

In [2]:
## dont run beyond 60 lack of computation - 92% ram 
## try 40 next

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import pathlib
import PIL
import os
import tarfile
import cv2
import numpy as np
import time

lower_age_limit = 1
upper_age_limit = 30

In [4]:
tar_files = ['part1.tar.gz', 'part2.tar.gz', 'part3.tar.gz']
output_folder = os.path.join(os.getcwd(), 'all_images')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

    for tar_file in tar_files:
        with tarfile.open(tar_file, 'r:gz') as tar:
            for member in tar.getmembers():
                # Remove the leading directory name from the member's name
                member.name = os.path.basename(member.name)
                tar.extract(member, output_folder)

In [5]:
data_dir = pathlib.Path('.//all_images')

In [6]:
image_count = len(list(data_dir.glob('**/*.jpg')))
print(image_count)

24106


In [7]:
age_images_dict = {}
ages_labels_dict = {}
bin_size = 1

for i in range(lower_age_limit, upper_age_limit, bin_size):
    age_bin = []
    for j in range(bin_size):
        age_bin.extend(list(data_dir.glob(f'{i+j}_*')))
        age_images_dict[i] = age_bin
        ages_labels_dict[i] = (i - lower_age_limit) // bin_size
    
    #age_images_dict[i] = list(data_dir.glob(f'{i}_*'))
    #ages_labels_dict[i] = i-1

# some ages are missing 

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def process_image(image):
    try:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        return resized_img
    except cv2.error as e:
        print(f"Error processing {image}: {str(e)}")
        return None

x, y = [], []

with ThreadPoolExecutor() as executor:
    futures = []
    for age_number, images in age_images_dict.items():
        for image in images:
            future = executor.submit(process_image, image)
            futures.append((future, age_number))

    for future, age_number in futures:
        result = future.result()
        if result is not None:
            x.append(result)
            y.append(ages_labels_dict[age_number])

x = np.array(x)
y = np.array(y)

In [9]:
np.savez( os.getcwd() + '/ age_image_data.npz', x, y)
del x, y, age_images_dict, ages_labels_dict, futures, result, images

# # run from here if saved .npz file

In [10]:
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras.metrics import Metric
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
from tensorflow.keras.models import Sequential
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from keras.callbacks import EarlyStopping

In [11]:
def macro_f1(y_true, y_pred):
    # Convert predicted probabilities to class labels
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)

    # Calculate the number of true positive, false positive, and false negative predictions for each class
    true_positives = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=0), tf.float32)
    false_positives = tf.cast(tf.math.count_nonzero((1 - y_true) * y_pred, axis=0), tf.float32)
    false_negatives = tf.cast(tf.math.count_nonzero(y_true * (1 - y_pred), axis=0), tf.float32)

    # Calculate precision and recall for each class
    precision = true_positives / (true_positives + false_positives + 1e-6)
    recall = true_positives / (true_positives + false_negatives + 1e-6)

    # Calculate the F1 score for each class
    f1 = 2 * precision * recall / (precision + recall + 1e-6)

    # Calculate the macro-averaged F1 score by taking the mean of the F1 scores for each class
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [19]:
with np.load(os.getcwd() + '/ age_image_data.npz', mmap_mode='c') as data:
    x = data['arr_0']
    y = data['arr_1']

num_classes = len(np.unique(y))

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)
del x, y

In [21]:
#sm = SMOTE(random_state=0)
sm = RandomUnderSampler(random_state=0)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_train_smote, y_train_smote = sm.fit_resample(X_train_reshaped, y_train)

X_train_smote = X_train_smote.reshape(X_train_smote.shape[0], X_train.shape[1], X_train.shape[2], 3)

unique_elements, counts = np.unique(y_train, return_counts=True)
print(f'Unique elements: {unique_elements}')
print(f'Counts: {counts}')
del X_train, y_train

Unique elements: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
Counts: [ 969  406  237  211  142  104  110  197  132  126   56  106   62  116
  128  188  106  191   79  219  270  311  323  630  564 1623  455  696
  434]


In [22]:
unique_elements, counts = np.unique(y_train_smote, return_counts=True)
print(f'Unique elements: {unique_elements}')
print(f'Counts: {counts}')

Unique elements: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28]
Counts: [56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56 56
 56 56 56 56 56]


In [23]:
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Rescaling(scale = 1./255, offset=0.0),
        layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(180, 180, 3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1)    
    ]
)
 
model = Sequential([
    
    data_augmentation,
    
    layers.Conv2D(64, 3, padding='valid', activation='elu', input_shape = (180,180,3)), # padding='same'
    layers.MaxPooling2D(), 
    
    layers.Conv2D(128, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(256, 3, padding='valid', activation='elu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(256, activation='elu'),
    layers.Dropout(0.2),
    layers.Dense(256, activation='elu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=[macro_f1])
es = EarlyStopping(monitor='macro_f1', verbose=0, mode = 'max')

In [25]:
model.fit(X_train_smote, y_train_smote, epochs=20, callbacks=[es])
print(model.evaluate(X_train_smote,y_train_smote)[1])
del X_train_smote, y_train_smote

Epoch 1/20
51/51 [==============================] - 92s 2s/step - loss: 3.7157 - macro_f1: 0.4852
Epoch 2/20
51/51 [==============================] - 18s 350ms/step - loss: 3.2095 - macro_f1: 0.4709
0.4709165692329407


In [26]:
X_test = X_test.reshape(X_test.shape[0], 180, 180, 3)
print(model.evaluate(X_test,y_test)[1])
del X_test,y_test

96/96 [==============================] - 34s 350ms/step - loss: 3.4363 - macro_f1: 0.4564
0.45641443133354187
